In [19]:
from alpha_vantage.timeseries import TimeSeries
api_key = 'WPZ50HWPV4VBK625'
import pandas as pd
import datetime as dt
import yfinance as yf
import requests
from bs4 import BeautifulSoup
import numpy as np
from stocktrends import Renko
import copy
import talib

# using yahoo finance

In [3]:
start = dt.datetime.today()-dt.timedelta(360)
end = dt.datetime.today()
quote = pd.DataFrame()
ohlcv = {}
symbol = []
sno = int(input("Number of stocks you eant to enter : "))
for _ in range(0,sno):
    symbol.append((input("Enter Ticker Symbol Here - Yahoo Finance : ")).upper())
    
def ticker1(symbol):
    for x in symbol:
        ohlcv[x] = yf.download(x,start,end)
    return ohlcv
        
table1 = ticker1(symbol)
table1

Number of stocks you eant to enter : 1
Enter Ticker Symbol Here - Yahoo Finance : itc.bo
[*********************100%***********************]  1 of 1 completed


{'ITC.BO':                   Open        High         Low       Close   Adj Close  \
 Date                                                                     
 2021-01-04  213.949997  216.000000  212.000000  213.399994  202.815735   
 2021-01-05  212.000000  213.250000  210.600006  211.500000  201.009979   
 2021-01-06  211.000000  211.600006  204.350006  205.449997  195.260040   
 2021-01-07  205.800003  207.550003  202.149994  202.750000  192.693970   
 2021-01-08  204.000000  206.000000  200.300003  201.550003  191.553482   
 ...                ...         ...         ...         ...         ...   
 2021-12-22  214.300003  214.300003  211.300003  212.050003  212.050003   
 2021-12-23  213.600006  218.350006  212.899994  217.300003  217.300003   
 2021-12-24  217.699997  220.449997  216.500000  218.000000  218.000000   
 2021-12-27  218.000000  218.250000  215.699997  217.550003  217.550003   
 2021-12-28  218.000000  220.550003  217.899994  219.949997  219.949997   
 
             

# using alpha vantage

In [4]:
ts = TimeSeries(key=api_key,output_format='pandas', indexing_type='date')
data = ts.get_daily(symbol='BSE:ITC',outputsize='full')[0] #0 is for the selecting dataframe form the tuple returned by fctn
data.columns = ["open","high","low","close","volume"]
data

,open,high,low,close,volume
date,,,,,
2021-12-28,218.0000,220.5500,217.9000,219.9500,786206.0
2021-12-27,218.0000,218.2500,215.7000,217.5500,786650.0
2021-12-24,217.7000,220.4500,216.5000,218.0000,1140304.0
2021-12-23,213.6000,218.3500,212.9000,217.3000,1486474.0
2021-12-22,214.3000,214.3000,211.3000,212.0500,1182123.0
...,...,...,...,...,...
2005-01-07,1280.5506,1302.0006,1272.0006,1298.3007,1896408.0
2005-01-06,1300.0005,1300.1007,1265.0007,1276.6005,2514399.0
2005-01-05,1323.0006,1323.0006,1281.1005,1294.0005,3260821.0


In [5]:
import time
ohlcv = {}
stocks = ["MSFT","AAPL","AMZN","GOOG","FB","CSCO"]
ts = TimeSeries(key=api_key,output_format='pandas', indexing_type='date')
ticker_count = 0
start_time = time.time()
for x in stocks:
    ohlcv[x] = ts.get_daily(symbol=x,outputsize='full')[0]
    ticker_count+=1
    if ticker_count==5:
        time.sleep(60- ((time.time()-start_time)%60.0))
        ticker_count=0  

In [6]:
ohlcv

{'MSFT':             1. open  2. high  3. low  4. close   5. volume
 date                                                      
 2021-12-27   335.46  342.480  335.43    342.45  19947029.0
 2021-12-23   332.75  336.390  332.73    334.69  19617848.0
 2021-12-22   328.30  333.605  325.75    333.20  24831493.0
 2021-12-21   323.29  327.730  319.80    327.29  24740597.0
 2021-12-20   320.05  322.800  317.57    319.91  28326532.0
 ...             ...      ...     ...       ...         ...
 1999-11-05    91.81   92.870   90.50     91.56  35083700.0
 1999-11-04    92.31   92.750   90.31     91.75  27119700.0
 1999-11-03    92.94   93.500   91.50     92.00  22258500.0
 1999-11-02    92.75   94.500   91.94     92.56  23174500.0
 1999-11-01    93.25   94.190   92.12     92.37  26630600.0
 
 [5575 rows x 5 columns],
 'AAPL':             1. open   2. high  3. low  4. close    5. volume
 date                                                        
 2021-12-27  177.085  180.4200  177.07    180.33   7

# using web scraping

In [7]:
tickers = ["AAPL","MSFT"] #list of tickers whose financial data needs to be extracted
financial_dir = {}

In [8]:
for ticker in tickers:
    #getting balance sheet data from yahoo finance for the given ticker
    temp_dir = {}
    url = 'https://finance.yahoo.com/quote/'+ticker+'/balance-sheet?p='+ticker
    headers={'User-Agent': "Mozilla/5.0"}
    page = requests.get(url, headers=headers)
    page_content = page.content
    soup = BeautifulSoup(page_content,'html.parser')
    tabl = soup.find_all("div", {"class" : "M(0) Whs(n) BdEnd Bdc($seperatorColor) D(itb)"})
    for t in tabl:
        rows = t.find_all("div", {"class" : "D(tbr) fi-row Bgc($hoverBgColor):h"})
        for row in rows:
            temp_dir[row.get_text(separator='|').split("|")[0]]=row.get_text(separator='|').split("|")[1]
    
    #getting income statement data from yahoo finance for the given ticker
    url = 'https://finance.yahoo.com/quote/'+ticker+'/financials?p='+ticker
    headers={'User-Agent': "Mozilla/5.0"}
    page = requests.get(url, headers=headers)
    page_content = page.content
    soup = BeautifulSoup(page_content,'html.parser')
    tabl = soup.find_all("div", {"class" : "M(0) Whs(n) BdEnd Bdc($seperatorColor) D(itb)"})
    for t in tabl:
        rows = t.find_all("div", {"class" : "D(tbr) fi-row Bgc($hoverBgColor):h"})
        for row in rows:
            temp_dir[row.get_text(separator='|').split("|")[0]]=row.get_text(separator='|').split("|")[1]
    
    #getting cashflow statement data from yahoo finance for the given ticker
    url = 'https://finance.yahoo.com/quote/'+ticker+'/cash-flow?p='+ticker
    headers={'User-Agent': "Mozilla/5.0"}
    page = requests.get(url, headers=headers)
    page_content = page.content
    soup = BeautifulSoup(page_content,'html.parser')
    tabl = soup.find_all("div", {"class" : "M(0) Whs(n) BdEnd Bdc($seperatorColor) D(itb)"})
    for t in tabl:
        rows = t.find_all("div", {"class" : "D(tbr) fi-row Bgc($hoverBgColor):h"})
        for row in rows:
            temp_dir[row.get_text(separator='|').split("|")[0]]=row.get_text(separator='|').split("|")[1]
    
    #getting key statistics data from yahoo finance for the given ticker
    url = 'https://finance.yahoo.com/quote/'+ticker+'/key-statistics?p='+ticker
    headers={'User-Agent': "Mozilla/5.0"}
    page = requests.get(url, headers=headers)
    page_content = page.content
    soup = BeautifulSoup(page_content,'html.parser')
    tabl = soup.findAll("table", {"class": "W(100%) Bdcl(c) "})
    for t in tabl:
        rows = t.find_all("tr")
        for row in rows:
            if len(row.get_text(separator='|').split("|")[0:2])>0:
                temp_dir[row.get_text(separator='|').split("|")[0]]=row.get_text(separator='|').split("|")[-1]    
    
    #combining all extracted information with the corresponding ticker
    financial_dir[ticker] = temp_dir

combined_financials = pd.DataFrame(financial_dir)
tickers = combined_financials.columns
for ticker in tickers:
    combined_financials = combined_financials[~combined_financials[ticker].str.contains("[a-z]").fillna(False)]

combined_financials

,AAPL,MSFT
Total Assets,"351,002,000","333,779,000"
Total Liabilities Net Minority Interest,"287,912,000","191,791,000"
Total Equity Gross Minority Interest,"63,090,000","141,988,000"
Total Capitalization,"172,196,000","192,062,000"
Common Stock Equity,"63,090,000","141,988,000"
...,...,...
Book Value Per Share,3.84,20.24
Levered Free Cash Flow,73.3B,49.82B
Capital Lease Obligations,NaN,"9,629,000"
Total Unusual Items Excluding Goodwill,NaN,"1,335,000"


# MACD implementation

In [11]:
tickers = ["MSFT","AAPL","GOOG"]
ohlcv_data = {}

for ticker in tickers:
    temp = yf.download(ticker,period='1mo',interval='15m')
    temp.dropna(how="any",inplace=True)
    ohlcv_data[ticker] = temp

def MACD(DF, a=12 ,b=26, c=9):
    """function to calculate MACD
       typical values a(fast moving average) = 12; 
                      b(slow moving average) =26; 
                      c(signal line ma window) =9"""
    df = DF.copy()
    df["ma_fast"] = df["Adj Close"].ewm(span=a, min_periods=a).mean()
    df["ma_slow"] = df["Adj Close"].ewm(span=b, min_periods=b).mean()
    df["macd"] = df["ma_fast"] - df["ma_slow"]
    df["signal"] = df["macd"].ewm(span=c, min_periods=c).mean()
    return df.loc[:,["macd","signal"]]

for ticker in ohlcv_data:
    ohlcv_data[ticker][["MACD","SIGNAL"]] = MACD(ohlcv_data[ticker], a=12 ,b=26, c=9)
    
ohlcv_data

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


# ATR implementation

In [ ]:
def ATR(DF, n=14):
    "function to calculate True Range and Average True Range"
    df = DF.copy()
    df["H-L"] = df["High"] - df["Low"]
    df["H-PC"] = abs(df["High"] - df["Adj Close"].shift(1))
    df["L-PC"] = abs(df["Low"] - df["Adj Close"].shift(1))
    df["TR"] = df[["H-L","H-PC","L-PC"]].max(axis=1, skipna=False)
    df["ATR"] = df["TR"].ewm(com=n, min_periods=n).mean()
    return df["ATR"]

for ticker in ohlcv_data:
    ohlcv_data[ticker]["ATR"] = ATR(ohlcv_data[ticker])

ohlcv_data

# Bollingers band

In [13]:
for ticker in tickers:
    temp = yf.download(ticker,period='1mo',interval='5m')
    temp.dropna(how="any",inplace=True)
    ohlcv_data[ticker] = temp

def Boll_Band(DF, n=14):
    "function to calculate Bollinger Band"
    df = DF.copy()
    df["MB"] = df["Adj Close"].rolling(n).mean()
    df["UB"] = df["MB"] + 2*df["Adj Close"].rolling(n).std(ddof=0)
    df["LB"] = df["MB"] - 2*df["Adj Close"].rolling(n).std(ddof=0)
    df["BB_Width"] = df["UB"] - df["LB"]
    return df[["MB","UB","LB","BB_Width"]]

for ticker in ohlcv_data:
    ohlcv_data[ticker][["MB","UB","LB","BB_Width"]] = Boll_Band(ohlcv_data[ticker])

ohlcv_data

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


{'MSFT':                                  Open        High         Low       Close  \
 Datetime                                                                    
 2021-11-29 09:30:00-05:00  334.940002  337.419312  334.739990  335.679993   
 2021-11-29 09:35:00-05:00  334.750000  336.369995  334.750000  336.179993   
 2021-11-29 09:40:00-05:00  336.000000  336.598999  335.529999  336.519989   
 2021-11-29 09:45:00-05:00  336.000000  337.049988  334.929993  336.430115   
 2021-11-29 09:50:00-05:00  336.750000  337.559998  336.750000  337.054993   
 ...                               ...         ...         ...         ...   
 2021-12-28 11:10:00-05:00  342.299988  342.390015  342.079987  342.083496   
 2021-12-28 11:15:00-05:00  342.089996  342.190002  341.920013  341.989990   
 2021-12-28 11:20:00-05:00  341.983704  342.160004  341.420013  341.576599   
 2021-12-28 11:25:00-05:00  341.559998  341.762604  341.339996  341.750000   
 2021-12-28 11:28:59-05:00  341.750000  341.750000  341.

# RSI implementation

In [16]:
for ticker in tickers:
    temp = yf.download(ticker,period='1mo',interval='5m')
    temp.dropna(how="any",inplace=True)
    ohlcv_data[ticker] = temp
    
def RSI(DF, n=14):
    "function to calculate RSI"
    df = DF.copy()
    df["change"] = df["Adj Close"] - df["Adj Close"].shift(1)
    df["gain"] = np.where(df["change"]>=0, df["change"], 0)
    df["loss"] = np.where(df["change"]<0, -1*df["change"], 0)
    df["avgGain"] = df["gain"].ewm(alpha=1/n, min_periods=n).mean()
    df["avgLoss"] = df["loss"].ewm(alpha=1/n, min_periods=n).mean()
    df["rs"] = df["avgGain"]/df["avgLoss"]
    df["rsi"] = 100 - (100/ (1 + df["rs"]))
    return df["rsi"]


for ticker in ohlcv_data:
    ohlcv_data[ticker]["RSI"] = RSI(ohlcv_data[ticker])
    
ohlcv_data

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


{'MSFT':                                  Open        High         Low       Close  \
 Datetime                                                                    
 2021-11-29 09:30:00-05:00  334.940002  337.419312  334.739990  335.679993   
 2021-11-29 09:35:00-05:00  334.750000  336.369995  334.750000  336.179993   
 2021-11-29 09:40:00-05:00  336.000000  336.598999  335.529999  336.519989   
 2021-11-29 09:45:00-05:00  336.000000  337.049988  334.929993  336.430115   
 2021-11-29 09:50:00-05:00  336.750000  337.559998  336.750000  337.054993   
 ...                               ...         ...         ...         ...   
 2021-12-28 11:15:00-05:00  342.089996  342.190002  341.920013  341.989990   
 2021-12-28 11:20:00-05:00  341.983704  342.160004  341.420013  341.576599   
 2021-12-28 11:25:00-05:00  341.559998  341.899994  341.339996  341.790009   
 2021-12-28 11:30:00-05:00  341.809998  341.929993  341.309998  341.459991   
 2021-12-28 11:35:40-05:00  341.339996  341.339996  341.

# renko implementation

In [18]:
tickers = ["AMZN","GOOG","MSFT"]
ohlcv_data = {}
hour_data = {}
renko_data = {}

# looping over tickers and storing OHLCV dataframe in dictionary
for ticker in tickers:
    temp = yf.download(ticker,period='1mo',interval='5m')
    temp.dropna(how="any",inplace=True)
    ohlcv_data[ticker] = temp
    temp = yf.download(ticker,period='1y',interval='1h')
    temp.dropna(how="any",inplace=True)
    hour_data[ticker] = temp
    
def ATR(DF, n=14):
    "function to calculate True Range and Average True Range"
    df = DF.copy()
    df["H-L"] = df["High"] - df["Low"]
    df["H-PC"] = abs(df["High"] - df["Adj Close"].shift(1))
    df["L-PC"] = abs(df["Low"] - df["Adj Close"].shift(1))
    df["TR"] = df[["H-L","H-PC","L-PC"]].max(axis=1, skipna=False)
    df["ATR"] = df["TR"].ewm(com=n, min_periods=n).mean()
    return df["ATR"]

def renko_DF(DF, hourly_df):
    "function to convert ohlc data into renko bricks"
    df = DF.copy()
    df.reset_index(inplace=True)
    df.drop("Close",axis=1,inplace=True)
    df.columns = ["date","open","high","low","close","volume"]
    df2 = Renko(df)
    df2.brick_size = 3*round(ATR(hourly_df,120).iloc[-1],0)
    renko_df = df2.get_ohlc_data() #if using older version of the library please use get_bricks() instead
    return renko_df

for ticker in ohlcv_data:
    renko_data[ticker] = renko_DF(ohlcv_data[ticker],hour_data[ticker])
    
renko_data

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


{'AMZN':                        date    open    high     low   close  uptrend
 0 2021-11-29 09:30:00-05:00  3393.0  3480.0  3393.0  3480.0     True
 1 2021-11-29 10:05:00-05:00  3480.0  3567.0  3480.0  3567.0     True
 2 2021-12-03 10:40:00-05:00  3480.0  3480.0  3393.0  3393.0    False,
 'GOOG':                        date    open    high     low   close  uptrend
 0 2021-11-29 09:30:00-05:00  2820.0  2880.0  2820.0  2880.0     True
 1 2021-12-07 10:10:00-05:00  2880.0  2940.0  2880.0  2940.0     True
 2 2021-12-20 09:30:00-05:00  2880.0  2880.0  2820.0  2820.0    False
 3 2021-12-22 11:55:00-05:00  2880.0  2940.0  2880.0  2940.0     True,
 'MSFT':                         date   open   high    low  close  uptrend
 0  2021-11-29 09:30:00-05:00  324.0  330.0  324.0  330.0     True
 1  2021-11-29 09:35:00-05:00  330.0  336.0  330.0  336.0     True
 2  2021-12-03 10:40:00-05:00  330.0  330.0  324.0  324.0    False
 3  2021-12-10 09:30:00-05:00  330.0  336.0  330.0  336.0     True
 4  2021-

# talib

In [21]:
talib.get_function_groups() #get a list of talib functions by group 

tickers = ["MSFT","AAPL","FB","AMZN","INTC", "CSCO","VZ","IBM","QCOM","LYFT"]

# Extract OHLCV data for the tickers
ohlc_tech = {} # directory with ohlc value for each stock            
ts = TimeSeries(key=api_key,output_format='pandas', indexing_type='date')

attempt = 0 # initializing passthrough variable
drop = [] # initializing list to store tickers whose close price was successfully extracted
while len(tickers) != 0 and attempt <=100:
    tickers = [j for j in tickers if j not in drop]
    for i in range(len(tickers)):
        try:
            ohlc_tech[tickers[i]] = ts.get_daily(symbol=tickers[i], outputsize='full')[0]
            ohlc_tech[tickers[i]].columns = ["Open","High","Low","Adj Close","Volume"]
            drop.append(tickers[i])      
        except:
            print(tickers[i]," :failed to fetch data...retrying")
            continue
    attempt+=1

 
tickers = ohlc_tech.keys() # redefine tickers variable after removing any tickers with corrupted data
ohlc_dict = copy.deepcopy(ohlc_tech) #create a copy of extracted data


# Apply talib functions on each dataframe - refer documentation at https://mrjbq7.github.io/ta-lib/doc_index.html
for ticker in tickers:
    # Calculate momentum indicators (e.g. MACD, ADX, RSI etc.) using talib
    ohlc_dict[ticker]["ADX"] = talib.ADX(ohlc_dict[ticker]["High"],
                                        ohlc_dict[ticker]["Low"],
                                        ohlc_dict[ticker]["Adj Close"],
                                        timeperiod=14)
    # Identify chart patterns (e.g. two crows, three crows, three inside, engulging pattern etc.)
    ohlc_dict[ticker]["3I"] = talib.CDL3WHITESOLDIERS(ohlc_dict[ticker]["Open"],
                                                 ohlc_dict[ticker]["High"],
                                                 ohlc_dict[ticker]["Low"],
                                                 ohlc_dict[ticker]["Adj Close"])
    
    # Statistical functions (e.g. beta, correlation etc.)
    ohlc_dict[ticker]["Beta"] = talib.BETA(ohlc_dict[ticker]["High"],
                                         ohlc_dict[ticker]["Low"],
                                         timeperiod=14)

ohlcv_dict

CSCO  :failed to fetch data...retrying
VZ  :failed to fetch data...retrying
IBM  :failed to fetch data...retrying
QCOM  :failed to fetch data...retrying
LYFT  :failed to fetch data...retrying
CSCO  :failed to fetch data...retrying
VZ  :failed to fetch data...retrying
IBM  :failed to fetch data...retrying
QCOM  :failed to fetch data...retrying
LYFT  :failed to fetch data...retrying
CSCO  :failed to fetch data...retrying
VZ  :failed to fetch data...retrying
IBM  :failed to fetch data...retrying
QCOM  :failed to fetch data...retrying


NameError: name 'ohlcv_dict' is not defined